# Gold Layer: Aggregations
The Gold Layer consists of aggregated tables to support business intelligence use cases and decision-making. Data in this layer is derived from the cleaned and transformed Silver layer, and is structured to provide insights.
This notebook focuses on 
- Customer Churn trends segmented by key dimensions such as `age_group`, `gender`, and `geography`.
- Aggregated metrics such a churn percentages, total customer counts, and group-specific statistics.



In [0]:
df_gold = spark.read.table("silver_customers")

In [0]:
from pyspark.sql.functions import *

In [0]:
display(df_gold.limit(10))

row_number,customer_id,surname,credit_score,geography,gender,age,tenure,Balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,complain,satisfaction_score,card_type,point_earned,time_ingested,age_group,credit_score_group,balance_group,estimated_salary_group,point_earned_group
433,15705521,Pisani,548,Germany,Female,33,0,101084.36,1,1,0,42749.85,0,0,5,DIAMOND,860,2025-07-31T02:50:30.321Z,25-34,poor,Medium,Low,751-1000
480,15797736,Smith,658,France,Male,29,4,80262.6,1,1,1,20612.82,0,0,2,GOLD,420,2025-07-31T02:50:30.321Z,25-34,fair,Medium,Low,251-500
1292,15679611,Andrews,734,Spain,Female,37,2,130404.92,1,0,0,34548.74,0,0,2,DIAMOND,434,2025-07-31T02:50:30.321Z,35-44,very good,Medium-High,Low,251-500
2900,15810944,Bryant,586,France,Female,35,7,0.0,2,1,0,70760.69,0,0,4,PLATINUM,481,2025-07-31T02:50:30.321Z,35-44,fair,No Balance,Medium-Low,251-500
4837,15728078,Yeh,609,France,Male,26,10,126392.18,1,0,1,43651.49,0,0,1,GOLD,494,2025-07-31T02:50:30.321Z,25-34,fair,Medium-High,Low,251-500
5502,15796496,Trevisani,631,France,Female,31,8,137687.72,1,1,0,190067.12,0,0,1,GOLD,498,2025-07-31T02:50:30.321Z,25-34,fair,High,High,251-500
5626,15663234,Bishop,508,France,Female,60,7,143262.05,1,1,1,129562.74,0,0,3,GOLD,727,2025-07-31T02:50:30.321Z,55-64,poor,High,Medium-High,501-750
5970,15579714,Pan,542,France,Female,29,7,0.0,2,0,1,196651.72,0,0,3,DIAMOND,761,2025-07-31T02:50:30.321Z,25-34,poor,No Balance,High,751-1000
6496,15640852,McGregor,617,Germany,Female,39,5,83348.89,3,1,0,7953.62,1,1,3,PLATINUM,994,2025-07-31T02:50:30.321Z,35-44,fair,Medium,Low,751-1000
8257,15768574,Anderson,671,Spain,Male,58,1,178713.98,1,1,1,21768.21,0,0,4,SILVER,953,2025-07-31T02:50:30.321Z,55-64,good,High,Low,751-1000


## Margin of Error
To ensure that these aggregated metrics are statistically reliable, we employ the concept of **Margin of Error** (MOE) and determine a minimum group size `n`.

The Margin of Error quantifies the precision of our calculated metrics for each customer segment. For instance, an average churn rate of 20% with a 2% MOE at a 95% confidence level means we are 95% confident the true churn rate falls between 18% and 22%. 

Calculating `n` gives us the minimum number of customers that must be present within an aggregated segment for its reported metrics. We derive `n` using a statistical formula that considers:

1. Our desired Confidence level 95%
2. Overall churn rate observed in the dataset

Margin of error = Z * √(p(1-p)/n)
<br>
Z = Z-score for selected confidence level (1.96 for 95% confidence)
<br>
p = sample proportion (churn rate)
<br>
n = sample size


In [0]:
import math 

churn_df = df_gold.agg(avg("exited"))
churn = churn_df.collect()[0][0]
print(f"churn rate: {churn}")
Z = 1.96
margin_of_error = 0.05
n = (Z**2) * churn * (1-churn) / (margin_of_error**2)
n = math.ceil(n)
print(f"minimum sample size: {n}")

churn rate: 0.2038
minimum sample size: 250


### Calculating Churn Rates by Account Balance

In [0]:
df_gold_balance = df_gold.groupBy("balance_group").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_balance = df_gold_balance.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_balance.orderBy(desc("churn_rate")))

balance_group,exited,not_exited,churn_rate
Medium-High,540,1441,0.27
Low,86,257,0.25
High,481,1609,0.23
Medium,431,1538,0.22
No Balance,500,3117,0.14


### Calculating Churn Rates by Credit Score

In [0]:
df_gold_credit = df_gold.groupBy("credit_score_group").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_credit = df_gold_credit.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_credit.orderBy(desc("churn_rate")))

credit_score_group,exited,not_exited,churn_rate
poor,420,1437,0.23
fair,708,2734,0.21
very good,194,739,0.21
excellent,272,1092,0.2
good,444,1960,0.18


### Calculating Churn Rates by Salary

In [0]:
df_gold_salary = df_gold.groupBy("estimated_salary_group").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_salary = df_gold_salary.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_salary.orderBy(desc("churn_rate")))


estimated_salary_group,exited,not_exited,churn_rate
High,556,2036,0.21
Medium-High,496,1969,0.2
Medium-Low,486,1961,0.2
Low,500,1996,0.2


### Calculating Churn Rates by Satisfaction Score

In [0]:
df_gold_satisfaction = df_gold.groupBy("satisfaction_score").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_satisfaction = df_gold_satisfaction.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_satisfaction.orderBy(desc("churn_rate")))

satisfaction_score,exited,not_exited,churn_rate
2,439,1575,0.22
4,414,1594,0.21
3,401,1641,0.2
1,387,1545,0.2
5,397,1607,0.2


### Calculating Churn Rates by Points Earned

In [0]:
df_gold_points = df_gold.groupBy("point_earned_group").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_points = df_gold_points.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_points.orderBy(desc("churn_rate")))

point_earned_group,exited,not_exited,churn_rate
501-750,668,2579,0.21
251-500,645,2520,0.2
0-250,86,345,0.2
751-1000,639,2518,0.2


### Calculating Churn Rates by Tenure

In [0]:
df_gold_tenure = df_gold.groupBy("tenure").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_tenure = df_gold_tenure.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_tenure.orderBy(desc("churn_rate")))

tenure,exited,not_exited,churn_rate
0,95,318,0.23
1,232,803,0.22
9,214,770,0.22
3,213,796,0.21
10,101,389,0.21
5,209,803,0.21
4,203,786,0.21
6,196,771,0.2
8,197,828,0.19
2,201,847,0.19


### Calculating Churn Rates by Card Type

In [0]:
df_gold_card_type = df_gold.groupBy("card_type").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_card_type = df_gold_card_type.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_card_type.orderBy(desc("churn_rate")))

card_type,exited,not_exited,churn_rate
DIAMOND,546,1961,0.22
SILVER,502,1994,0.2
PLATINUM,508,1987,0.2
GOLD,482,2020,0.19


### Calculating Churn Rates by Credit Card Ownership

In [0]:
df_gold_credit_card = df_gold.groupBy("has_cr_card").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_credit_card = df_gold_credit_card.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_credit_card.orderBy(desc("churn_rate")))

has_cr_card,exited,not_exited,churn_rate
0,613,2332,0.21
1,1425,5630,0.2


### Calculating Churn Rates by Geography

In [0]:
df_gold_geo = df_gold.groupBy("geography").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_geo = df_gold_geo.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_geo)

geography,exited,not_exited,churn_rate
Germany,814,1695,0.32
Spain,413,2064,0.17
France,811,4203,0.16


### Calculating Churn Rates by Age 

In [0]:
df_gold_age = df_gold.groupBy("age_group").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_age = df_gold_age.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_age.orderBy(desc("churn_rate")))


age_group,exited,not_exited,churn_rate
55-64,299,301,0.5
45-54,702,756,0.48
35-44,704,3277,0.18
65+,43,239,0.15
18-24,40,417,0.09
25-34,250,2972,0.08


### Calculating Churn Rates by Number of Products

In [0]:
df_gold_number_of_products = df_gold.groupBy("num_of_products").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_number_of_products = df_gold_number_of_products.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_number_of_products.orderBy(desc("churn_rate")))

num_of_products,exited,not_exited,churn_rate
4,60,0,1.0
3,220,46,0.83
1,1409,3675,0.28
2,349,4241,0.08


### Calculating Churn Rates by Account Activity

In [0]:
df_gold_active = df_gold.groupBy("is_active_member").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_active = df_gold_active.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_active.orderBy(desc("churn_rate")))


is_active_member,exited,not_exited,churn_rate
0,1303,3546,0.27
1,735,4416,0.14


### Calculating Churn Rates by Customer Complaint

In [0]:
df_gold_complain = df_gold.groupBy("complain").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_complain = df_gold_complain.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_complain.orderBy(desc("churn_rate")))


complain,exited,not_exited,churn_rate
1,2034,10,1.0
0,4,7952,0.0


### Calculating Churn Rates by Gender

In [0]:
df_gold_gender = df_gold.groupBy("gender").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_gender = df_gold_gender.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
display(df_gold_gender.orderBy(desc("churn_rate")))

gender,exited,not_exited,churn_rate
Female,1139,3404,0.25
Male,899,4558,0.16


### Columns of significance for churn rate
- geography
- age groups
- number of products
- is active
- gender



## Dashboard Tables

### Churn Rates by Age and Gender

In [0]:
df_gold_age_gender = df_gold.groupBy("age_group", "gender").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_age_gender = df_gold_age_gender.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
#df_gold_age_gender = df_gold_age_gender.filter((col("exited") + col("not_exited")) >= (n))
df_gold_age_gender = df_gold_age_gender.withColumn("age_gender", concat(col("age_group"), lit(" "), col("gender")))
display(df_gold_age_gender.orderBy(desc("churn_rate")))

age_group,gender,exited,not_exited,churn_rate,age_gender
55-64,Female,172,126,0.58,55-64 Female
45-54,Female,390,333,0.54,45-54 Female
45-54,Male,312,423,0.42,45-54 Male
55-64,Male,127,175,0.42,55-64 Male
35-44,Female,385,1386,0.22,35-44 Female
65+,Female,25,103,0.2,65+ Female
35-44,Male,319,1891,0.14,35-44 Male
65+,Male,18,136,0.12,65+ Male
18-24,Female,23,180,0.11,18-24 Female
25-34,Female,144,1276,0.1,25-34 Female


### Churn Rates by Age and Products

In [0]:
df_gold_age_products = df_gold.groupBy("age_group", "num_of_products").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_age_products = df_gold_age_products.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
df_gold_age_products = df_gold_age_products.filter((col("exited") + col("not_exited")) >= (n))
df_gold_age_products = df_gold_age_products.withColumn("age_products", concat(col("age_group"), lit(" "), col("num_of_products")))
display(df_gold_age_products.orderBy(desc("churn_rate")))

age_group,num_of_products,exited,not_exited,churn_rate,age_products
45-54,1,517,358,0.59,45-54 1
55-64,1,197,156,0.56,55-64 1
35-44,1,463,1503,0.24,35-44 1
45-54,2,98,397,0.2,45-54 2
25-34,1,179,1343,0.12,25-34 1
35-44,2,133,1753,0.07,35-44 2
25-34,2,40,1611,0.02,25-34 2


### Churn Rates by Age and Activity

In [0]:
df_gold_age_active = df_gold.groupBy("age_group", "is_active_member").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_age_active = df_gold_age_active.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
df_gold_age_active = df_gold_age_active.filter((col("exited") + col("not_exited")) >= (n))
df_gold_age_active = df_gold_age_active.withColumn("age_active", concat(col("age_group"), lit(" "), col("is_active_member")))
display(df_gold_age_active.orderBy(desc("churn_rate")))

age_group,is_active_member,exited,not_exited,churn_rate,age_active
45-54,0,459,300,0.6,45-54 0
45-54,1,243,456,0.35,45-54 1
55-64,1,110,285,0.28,55-64 1
35-44,0,449,1600,0.22,35-44 0
35-44,1,255,1677,0.13,35-44 1
25-34,0,153,1427,0.1,25-34 0
25-34,1,97,1545,0.06,25-34 1


### Churn Rates By Geography and Activity

In [0]:
df_gold_geo_active = df_gold.groupBy("geography", "is_active_member").agg(count(when(col("exited")==1, 1)).alias("exited"), count(when(col("exited")==0, 1)).alias("not_exited"))
df_gold_geo_active = df_gold_geo_active.withColumn("churn_rate", round(col("exited")/(col("exited")+col("not_exited")),2))
df_gold_geo_active = df_gold_geo_active.filter((col("exited") + col("not_exited")) >= (n))
df_gold_geo_active = df_gold_geo_active.withColumn("geo_active", concat(col("geography"), lit(" "), col("is_active_member")))
display(df_gold_geo_active.orderBy(desc("churn_rate")))

geography,is_active_member,exited,not_exited,churn_rate,geo_active
Germany,0,518,743,0.41,Germany 0
Germany,1,296,952,0.24,Germany 1
Spain,0,272,893,0.23,Spain 0
France,0,513,1910,0.21,France 0
France,1,298,2293,0.12,France 1
Spain,1,141,1171,0.11,Spain 1


### Save the Tables

In [0]:
df_gold_age_gender.write.format("delta").mode("overwrite").saveAsTable("gold_age_gender")
df_gold_age_products.write.format("delta").mode("overwrite").saveAsTable("gold_age_products")
df_gold_age_active.write.format("delta").mode("overwrite").saveAsTable("gold_age_active")
df_gold_geo_active.write.format("delta").mode("overwrite").saveAsTable("gold_geo_active")
df_gold_age.write.format("delta").mode("overwrite").saveAsTable("gold_age")
df_gold_geo.write.format("delta").mode("overwrite").saveAsTable("gold_geo")